In [2]:
# import relevant modules/packages
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from pgmpy.estimators import K2Score, BicScore#, BDeuScore
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, BayesianEstimator

from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

from pgmpy.estimators import BdeuScore, K2Score, BicScore
#from pgmpy.models import BayesianModel

from pgmpy.estimators import HillClimbSearch

from pgmpy.estimators import BayesianEstimator

from pgmpy.inference import VariableElimination

In [3]:
# read data
Data=pd.read_csv("DataInputFinal.csv")

In [4]:
import warnings
warnings.filterwarnings("ignore")

Data['Asthma_binned'] = pd.cut(Data['Asthma Hospitalizations (Adults) by NTA'], bins=5, labels=False)
Data['FamiliesIncome_binned'] = pd.cut(Data['Families with income < $10,000'], bins=5, labels=False)
Data['11th_binned'] = pd.cut(Data['Number of persons 25+ yrs in 11th grade'], bins=5, labels=False)
Data['ChildrenPoverty'] = pd.cut(Data['Children under 5 years old in Poverty'], bins=5, labels=False)
Data['PercentBlack_binned'] = pd.cut(Data['Percent Black Alone'], bins=5, labels=False)
Data['10th_binned'] = pd.cut(Data['Number of persons 25+ yrs in 10th grade'], bins=5, labels=False)
Data['Poverty_binned'] = pd.cut(Data['Poverty'], bins=5, labels=False)
Data['Families125K_binned'] = pd.cut(Data['Families with income between $125,000-$149,999'], bins=5, labels=False)
Data['Unemployment_binned'] = pd.cut(Data['Unemployment'], bins=5, labels=False)
Data['Families150K_binned'] = pd.cut(Data['Families with income between $150,000-$199,999'], bins=5, labels=False)
Data['PercentAsian_binned'] = pd.cut(Data['Percent Asian Alone'], bins=5, labels=False)



Data_cutfeatures = Data.iloc[:,64:]


Data_cutfeatures.head()

,Asthma_binned,FamiliesIncome_binned,11th_binned,ChildrenPoverty,PercentBlack_binned,10th_binned,Poverty_binned,Families125K_binned,Unemployment_binned,Families150K_binned,PercentAsian_binned
0,1,0,1,1,1,1,0,2,2,2,0
1,0,0,0,0,0,1,0,4,0,4,0
2,0,0,1,0,0,1,0,3,0,4,0
3,0,0,1,1,0,1,1,2,1,1,1
4,0,0,0,0,0,1,0,2,0,3,3


In [5]:
# train test split
train = Data_cutfeatures.iloc[0:125,:]
test = Data_cutfeatures.iloc[126:-1,:]

In [6]:
train.columns

Index([u'Asthma_binned', u'FamiliesIncome_binned', u'11th_binned',
       u'ChildrenPoverty', u'PercentBlack_binned', u'10th_binned',
       u'Poverty_binned', u'Families125K_binned', u'Unemployment_binned',
       u'Families150K_binned', u'PercentAsian_binned'],
      dtype='object')

In [7]:
# cutfeatures = ['Asthma Hospitalizations (Adults) by NTA', 'Families with income < $10,000',
#                'Number of persons 25+ yrs in 11th grade','Children under 5 years old in Poverty',
#                'Percent Black Alone','Number of persons 25+ yrs in 10th grade','Poverty',
#                'Families with income between $125,000-$149,999','Unemployment',
#                'Families with income between $150,000-$199,999','Percent Asian Alone']

In [8]:
# train_cutfeatures = train[cutfeatures]
# test_cutfeatures = test[cutfeatures]

In [9]:
# train_cutfeatures.head()

In [10]:
# import warnings
# warnings.filterwarnings("ignore")

# train_cutfeatures['Asthma_binned'] = pd.cut(train_cutfeatures['Asthma Hospitalizations (Adults) by NTA'], bins=5, labels=False)
# train_cutfeatures['FamiliesIncome_binned'] = pd.cut(train_cutfeatures['Families with income < $10,000'], bins=5, labels=False)
# train_cutfeatures['11th_binned'] = pd.cut(train_cutfeatures['Number of persons 25+ yrs in 11th grade'], bins=5, labels=False)
# train_cutfeatures['ChildrenPoverty'] = pd.cut(train_cutfeatures['Children under 5 years old in Poverty'], bins=5, labels=False)
# train_cutfeatures['PercentBlack_binned'] = pd.cut(train_cutfeatures['Percent Black Alone'], bins=5, labels=False)
# train_cutfeatures['10th_binned'] = pd.cut(train_cutfeatures['Number of persons 25+ yrs in 10th grade'], bins=5, labels=False)
# train_cutfeatures['Poverty_binned'] = pd.cut(train_cutfeatures['Poverty'], bins=5, labels=False)
# train_cutfeatures['Families125K_binned'] = pd.cut(train_cutfeatures['Families with income between $125,000-$149,999'], bins=5, labels=False)
# train_cutfeatures['Unemployment_binned'] = pd.cut(train_cutfeatures['Unemployment'], bins=5, labels=False)
# train_cutfeatures['Families150K_binned'] = pd.cut(train_cutfeatures['Families with income between $150,000-$199,999'], bins=5, labels=False)
# train_cutfeatures['PercentAsian_binned'] = pd.cut(train_cutfeatures['Percent Asian Alone'], bins=5, labels=False)



# train_cutfeatures = train_cutfeatures.iloc[:,11:]


# train_cutfeatures.head()

In [11]:
train.columns

Index([u'Asthma_binned', u'FamiliesIncome_binned', u'11th_binned',
       u'ChildrenPoverty', u'PercentBlack_binned', u'10th_binned',
       u'Poverty_binned', u'Families125K_binned', u'Unemployment_binned',
       u'Families150K_binned', u'PercentAsian_binned'],
      dtype='object')

In [12]:
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = BayesianModel(hc.estimate().edges())
print(best_model.edges())

[('11th_binned', 'Asthma_binned'), ('10th_binned', '11th_binned'), ('Asthma_binned', 'PercentBlack_binned'), ('Asthma_binned', 'Unemployment_binned'), ('Asthma_binned', 'ChildrenPoverty'), ('ChildrenPoverty', 'Poverty_binned'), ('ChildrenPoverty', 'Families150K_binned'), ('Families150K_binned', 'Families125K_binned'), ('Poverty_binned', 'FamiliesIncome_binned')]


In [13]:
from pgmpy.estimators import BayesianEstimator
# your code here
# best_model = BayesianModel([('11th_binned', 'Asthma_binned'), ('10th_binned', '11th_binned'), ('Asthma_binned', 'PercentBlack_binned'),
#                             ('Asthma_binned', 'Unemployment_binned'), ('Asthma_binned', 'ChildrenPoverty'),
#                             ('ChildrenPoverty', 'Poverty_binned'),
#                             ('ChildrenPoverty', 'Families150K_binned'),
#                             ('Families150K_binned', 'Families125K_binned'),
#                             ('Poverty_binned', 'FamiliesIncome_binned')])
best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
for cpd in best_model.get_cpds():
    print('CPD of {variable}:'.format(variable=cpd.variable))
    print(cpd)
print best_model.local_independencies(best_model.nodes())

CPD of PercentBlack_binned:
+------------------------+----------------------+---------------------+---------------------+---------------------+---------------------+
| Asthma_binned          | Asthma_binned(0)     | Asthma_binned(1)    | Asthma_binned(2)    | Asthma_binned(3)    | Asthma_binned(4)    |
+------------------------+----------------------+---------------------+---------------------+---------------------+---------------------+
| PercentBlack_binned(0) | 0.8873239436619719   | 0.3235294117647059  | 0.3333333333333333  | 0.16666666666666666 | 0.08333333333333333 |
+------------------------+----------------------+---------------------+---------------------+---------------------+---------------------+
| PercentBlack_binned(1) | 0.028169014084507043 | 0.20588235294117646 | 0.2                 | 0.4444444444444444  | 0.5833333333333334  |
+------------------------+----------------------+---------------------+---------------------+---------------------+---------------------+
| Perc

In [14]:
# your answers here -- C

from collections import defaultdict

def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    for index, data_point in data.iterrows():
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

train_x = train[['FamiliesIncome_binned','11th_binned','ChildrenPoverty', 'PercentBlack_binned', '10th_binned',
                'Poverty_binned', 'Families125K_binned', 'Unemployment_binned','Families150K_binned']]
# #predicted_train = best_model.predict(train_x) should work, but is buggy in the current version
# predicted_train = temp_predict(best_model, train_x)
# print "In sample:",(train.loc[:,'A']==predicted_train['A']).mean()

test_x = test[['FamiliesIncome_binned','11th_binned','ChildrenPoverty', 'PercentBlack_binned', '10th_binned',
                'Poverty_binned', 'Families125K_binned', 'Unemployment_binned','Families150K_binned']]
#predicted_test = best_model.predict(test_x) #should work, but is buggy in the current version
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(test.loc[:,'Asthma_binned']==predicted_test['Asthma_binned']).mean()

Out of sample: 0.6346153846153846
